In [1]:
import numpy as np
import pandas as pd

In [2]:
movies=pd.read_csv("/content/drive/MyDrive/PROJECTS/Movie Recommender/tmdb_5000_movies.csv/tmdb_5000_movies.csv")
credits=pd.read_csv("/content/drive/MyDrive/PROJECTS/Movie Recommender/tmdb_5000_credits.csv/tmdb_5000_credits.csv")

In [3]:
movies=movies.merge(credits,on="title")

In [4]:
#we keep only specific columns which will be useful in content based prediction
movies=movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [5]:
#we check weather there are any cells with NA and deal with it
movies.isnull().sum()
movies.dropna(inplace=True)
movies.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [6]:
#we check if there are any duplicate rows
movies.duplicated().sum()
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [7]:
# no we want to convert entire data frame to three columns
#Movie_id Overview Movie_tags

In [8]:
import ast
def convert(obj):
  l=[]
  for i in ast.literal_eval(obj): #obj is string so ast is used to convert string to list
    l.append(i['name'])
  return l

In [9]:
movies['genres']=movies['genres'].apply(convert)

In [10]:
movies['keywords']=movies['keywords'].apply(convert)

In [11]:
def convert_cast(obj):
  l=[]
  counter=0
  for i in ast.literal_eval(obj): #obj is string so ast is used to convert string to list
    if counter!=3:
      l.append(i['name'])
      counter+=1
    else:
      break
  return l

In [12]:
movies['cast']=movies['cast'].apply(convert_cast)

In [13]:
def convert_crew(obj):
  l=[]
  for i in ast.literal_eval(obj):
    if i['job']=='Director':
      l.append(i['name'])
      break
  return l

In [14]:
movies['crew']=movies['crew'].apply(convert_crew)

In [15]:
movies['overview']=movies['overview'].apply(lambda x:x.split())#converting string to list so that it can be combined with other columns

In [16]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","")for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","")for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","")for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","")for i in x])

In [17]:
movies['tags']=movies['overview']+movies['genres']+movies['crew']+movies['cast']+movies['keywords']
movies.iloc[0].tags

['In',
 'the',
 '22nd',
 'century,',
 'a',
 'paraplegic',
 'Marine',
 'is',
 'dispatched',
 'to',
 'the',
 'moon',
 'Pandora',
 'on',
 'a',
 'unique',
 'mission,',
 'but',
 'becomes',
 'torn',
 'between',
 'following',
 'orders',
 'and',
 'protecting',
 'an',
 'alien',
 'civilization.',
 'Action',
 'Adventure',
 'Fantasy',
 'ScienceFiction',
 'JamesCameron',
 'SamWorthington',
 'ZoeSaldana',
 'SigourneyWeaver',
 'cultureclash',
 'future',
 'spacewar',
 'spacecolony',
 'society',
 'spacetravel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alienplanet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'loveaffair',
 'antiwar',
 'powerrelations',
 'mindandsoul',
 '3d']

In [18]:
data=movies[['movie_id','title','tags']]
data['tags']=data['tags'].apply(lambda x:" ".join(x))
data['tags']=data['tags'].apply(lambda x:x.lower())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [19]:
data['tags'][0]

'in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy sciencefiction jamescameron samworthington zoesaldana sigourneyweaver cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d'

In [30]:
!pip install nltk
import nltk
from nltk.stem.porter import PorterStemmer 
ps=PorterStemmer()

In [31]:
def stem_text(text):
  y=[]
  for i in text.split():
    y.append(ps.stem(i))
  return " ".join(y)

In [34]:
data['tags']=data['tags'].apply(stem_text)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [35]:
from sklearn.feature_extracti  on.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')

In [36]:
vectors=cv.fit_transform(data['tags']).toarray()

In [38]:
 from sklearn.metrics.pairwise import cosine_similarity


In [40]:
similarity=cosine_similarity(vectors)

In [48]:
def prediction(movie):
  movie_index=data[data['title']==movie].index[0]
  distance=similarity[movie_index]
  movie_list=sorted(list(enumerate(distance)),reverse=True,key=lambda x:x[1])[1:6]
  for i in movie_list:
    print(data.iloc[i[0]].title)

In [50]:
prediction('Titanic')

The Notebook
Under the Same Moon
Ghost Ship
The Bounty
Pirates of the Caribbean: On Stranger Tides


In [52]:
import pickle
pickle.dump(data,open('movie_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))